## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [78]:
# Your code here - remember to use markdown cells for comments as well!

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [79]:
pd.set_option('display.width', 1000)
mg = pd.read_csv("zippedData/bom.movie_gross.csv")
nm_bcs = pd.read_csv("zippedData/imdb.name.basics.csv")
tle_bcs = pd.read_csv("zippedData/imdb.title.basics.csv")
tle_aka = pd.read_csv("zippedData/imdb.title.akas.csv")
tle_ppl = pd.read_csv("zippedData/imdb.title.principals.csv")
rt_info = pd.read_csv("zippedData/rt.movie_info.tsv", sep="\t")
rt_rvw = pd.read_csv("zippedData/rt.reviews.tsv", encoding= 'unicode_escape',  sep="\t")
imdb_mv = pd.read_csv("zippedData/tmdb.movies.csv")
mv_bdg = pd.read_csv("zippedData/tn.movie_budgets.csv")



In [80]:
##tle_bcs[tle_bcs['primary_title'] == 'Toy Story 3']
imdb_mv[imdb_mv['title'] == 'Toy Story 3']

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
7,7,"[16, 10751, 35]",10193,en,Toy Story 3,24.445,2010-06-17,Toy Story 3,7.7,8340


In [81]:
mg.columns


Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')

In [82]:
mg.index

RangeIndex(start=0, stop=3387, step=1)

In [83]:
mg

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [84]:
mg['domestic_gross']

0       415000000.0
1       334200000.0
2       296000000.0
3       292600000.0
4       238700000.0
           ...     
3382         6200.0
3383         4800.0
3384         2500.0
3385         2400.0
3386         1700.0
Name: domestic_gross, Length: 3387, dtype: float64

In [85]:
##'foreign_gross' series has both strings and floats
## The strings have a comma in them
##First I converted the whole series into strings
mg['foreign_gross'] = mg['foreign_gross'].apply(lambda x: str(x))

In [86]:
##Next I removed the commas
mg['foreign_gross'] = mg['foreign_gross'].apply(lambda x: x.replace(',', ''))

In [87]:
mg['foreign_gross'] = mg['foreign_gross'].apply(lambda x: float(x))

In [88]:
mg['total_gross'] = mg['domestic_gross'] + mg['foreign_gross']

In [89]:
mg['total_gross']

0       1.067000e+09
1       1.025500e+09
2       9.603000e+08
3       8.283000e+08
4       7.526000e+08
            ...     
3382             NaN
3383             NaN
3384             NaN
3385             NaN
3386             NaN
Name: total_gross, Length: 3387, dtype: float64

In [90]:
mg.dropna(axis=0, subset=['total_gross'], inplace=True)

In [91]:
mg.isnull().sum()

title             0
studio            2
domestic_gross    0
foreign_gross     0
year              0
total_gross       0
dtype: int64

In [92]:
mg

,title,studio,domestic_gross,foreign_gross,year,total_gross
0,Toy Story 3,BV,415000000.0,652000000.0,2010,1.067000e+09
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,1.025500e+09
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,9.603000e+08
3,Inception,WB,292600000.0,535700000.0,2010,8.283000e+08
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,7.526000e+08
...,...,...,...,...,...,...
3275,I Still See You,LGF,1400.0,1500000.0,2018,1.501400e+06
3286,The Catcher Was a Spy,IFC,725000.0,229000.0,2018,9.540000e+05
3309,Time Freak,Grindstone,10000.0,256000.0,2018,2.660000e+05
3342,Reign of Judges: Title of Liberty - Concept Short,Darin Southa,93200.0,5200.0,2018,9.840000e+04


In [93]:
mg['total_gross']

0       1.067000e+09
1       1.025500e+09
2       9.603000e+08
3       8.283000e+08
4       7.526000e+08
            ...     
3275    1.501400e+06
3286    9.540000e+05
3309    2.660000e+05
3342    9.840000e+04
3353    7.320000e+04
Name: total_gross, Length: 2009, dtype: float64

In [76]:
tle_bcs[tle_bcs['primary_title'] == 'Inception']

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
7066,tt1375666,Inception,Inception,2010,148.0,"Action,Adventure,Sci-Fi"


In [94]:
movies = (mg.merge(tle_bcs, left_on='title', right_on='primary_title')
          .reindex(columns=['title', 'studio', 'total_gross', 'year', 'runtime_minutes', 'genres']))

In [95]:
movies

,title,studio,total_gross,year,runtime_minutes,genres
0,Toy Story 3,BV,1.067000e+09,2010,103.0,"Adventure,Animation,Comedy"
1,Inception,WB,8.283000e+08,2010,148.0,"Action,Adventure,Sci-Fi"
2,Shrek Forever After,P/DW,7.526000e+08,2010,93.0,"Adventure,Animation,Comedy"
3,The Twilight Saga: Eclipse,Sum.,6.985000e+08,2010,124.0,"Adventure,Drama,Fantasy"
4,Iron Man 2,Par.,6.239000e+08,2010,124.0,"Action,Adventure,Sci-Fi"
...,...,...,...,...,...,...
2014,Bilal: A New Breed of Hero,VE,2.191000e+06,2018,105.0,"Action,Adventure,Animation"
2015,I Still See You,LGF,1.501400e+06,2018,98.0,"Fantasy,Thriller"
2016,The Catcher Was a Spy,IFC,9.540000e+05,2018,98.0,"Biography,Drama,War"
2017,Time Freak,Grindstone,2.660000e+05,2018,104.0,"Comedy,Drama,Romance"


In [96]:
movies = (movies.merge(imdb_mv, left_on='title', right_on='title')
          .reindex(columns=['title', 'studio', 'total_gross', 'year', 'runtime_minutes', 'genres', 'popularity']))

In [97]:
movies

,title,studio,total_gross,year,runtime_minutes,genres,popularity
0,Toy Story 3,BV,1.067000e+09,2010,103.0,"Adventure,Animation,Comedy",24.445
1,Inception,WB,8.283000e+08,2010,148.0,"Action,Adventure,Sci-Fi",27.920
2,Shrek Forever After,P/DW,7.526000e+08,2010,93.0,"Adventure,Animation,Comedy",15.041
3,The Twilight Saga: Eclipse,Sum.,6.985000e+08,2010,124.0,"Adventure,Drama,Fantasy",20.340
4,Iron Man 2,Par.,6.239000e+08,2010,124.0,"Action,Adventure,Sci-Fi",28.515
...,...,...,...,...,...,...,...
2340,Bilal: A New Breed of Hero,VE,2.191000e+06,2018,105.0,"Action,Adventure,Animation",2.707
2341,I Still See You,LGF,1.501400e+06,2018,98.0,"Fantasy,Thriller",12.867
2342,The Catcher Was a Spy,IFC,9.540000e+05,2018,98.0,"Biography,Drama,War",9.727
2343,Time Freak,Grindstone,2.660000e+05,2018,104.0,"Comedy,Drama,Romance",1.604


In [60]:
tle_bcs[tle_bcs['primary_title'] == 'Toy Story 3']

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
104,tt0435761,Toy Story 3,Toy Story 3,2010,103.0,"Adventure,Animation,Comedy"
